**<center><font size="15">Water Meter Dataset Initial Analysis</font><center>**

<center><font size="5">Prepared by Kristen Davis for WSO Interview Round 1</font><center>

# Libraries and Data Initialization 
Libraries, packages, style guide and data used in analysis 

In [120]:
#libraries
import pandas as pd  
import plotly.graph_objects as go #visualization
import plotly.express as px #visualization

In [118]:
#color scheme
colors = ['rgb(26, 118, 255)', 'rgb(75, 117, 156)', 'rgb(135, 206, 235)'] 

#font family 
font_family = 'Georgia'

In [23]:
#primary dataset 
df = pd.read_csv("Data/WSO_Analyst_Example_Dataset.csv") 

#sanity check 
df.head()

,meterID,testYear,installationYear,meterSize,meterMake,accuracy
0,35826455,2019,1991,5/8,NEPTUNE,98.72%
1,85381440,2018,2007,5/8,NEPTUNE,0.00%
2,43165289,2019,1997,5/8,NEPTUNE,98.95%
3,42947481,2019,1997,5/8,NEPTUNE,99.86%
4,18432485,2019,2002,5/8,BADGER,99.85%


In [76]:
#customre dataset 
cdf = pd.read_csv("Data/Summary_Customer_Usage.csv") 

#sanity check 
cdf.head()

,Meter Make,Total 2019 Recorded Usage (CCF),Count of Meters Currently in Service
0,Badger,"28,576,886","172,745"
1,Neptune,"23,316,556","166,182"
2,Trident,"2,094,991","13,897"


# Meta EDA & Data Manipulation
Generalized metrics, statistics, and column based data manipulation (as needed)

In [24]:
#descriptive statistics
df.describe()

,testYear,installationYear
count,1962.000000,1962.000000
mean,2018.537207,1997.147299
std,0.522704,14.305625
min,2018.000000,1941.000000
25%,2018.000000,1994.000000
50%,2019.000000,2000.000000
75%,2019.000000,2006.000000
max,2020.000000,2019.000000


In [25]:
#data types  
df.dtypes

meterID             object
testYear             int64
installationYear     int64
meterSize           object
meterMake           object
accuracy            object
dtype: object

*Based on the initial describe results returning only 2 columns when I expected to see 4 (testYear, installationYear, meterSize & accuracy) I know that one of the first tasks needed will be to assign meterSize and accuracy to numerical column types. Dtypes confirms this.*

In [26]:
#current types
df['meterSize'].unique()

array(['5/8', '5/8"', '1"', '1', '5/8" ', '3/4'], dtype=object)

In [33]:
#replace meterSize as a decimal 
df['meterSize'] = df['meterSize'].apply(lambda x: x.replace("5/8", "0.625"))
df['meterSize'] = df['meterSize'].apply(lambda x: x.replace("3/4", "0.75")) 

#drop extra " 
df['meterSize'] = df['meterSize'].apply(lambda x: x.strip('" '))

#cast column as float 
df['meterSize'] = df['meterSize'].astype(float)

*The data dictionary notes measurements in inches. I will cast current fractions as ints 0.625 = 5/8th / 0.75 = 3/4th / 1 = 1*

In [36]:
#current types
df['accuracy'].unique()

array(['98.72%', '0.00%', '98.95%', '99.86%', '99.85%', '98.66%',
       '100.58%', '97.92%', '100.13%', '99.83%', '96.65%', '92.70%',
       '98.80%', '84.28%', '98.93%', '98.23%', '97.72%', '80.24%',
       '100.65%', '98.99%', '95.94%', '99.42%', '97.11%', '98.90%',
       '97.60%', '99.00%', '99.77%', '99.92%', '100.04%', '98.61%',
       '99.17%', '99.87%', '4.45%', '97.25%', '81.50%', '99.74%',
       '97.88%', '100.10%', '99.78%', '100.44%', '94.13%', '99.19%',
       '99.71%', '99.39%', '99.28%', '99.58%', '100.87%', '98.62%',
       '96.08%', '100.05%', '100.11%', '99.31%', '98.78%', '98.29%',
       '98.48%', '99.29%', '98.76%', '101.40%', '99.18%', '99.44%',
       '100.62%', '99.24%', '99.91%', '100.22%', '98.49%', '99.88%',
       '97.86%', '98.88%', '100.29%', '99.52%', '99.38%', '99.04%',
       '99.32%', '90.47%', '96.73%', '99.80%', '99.45%', '99.25%',
       '99.47%', '94.68%', '100.01%', '98.41%', '97.03%', '99.06%',
       '84.33%', '99.82%', '100.56%', '100.06%', '

In [38]:
#drop % 
df['accuracy'] = df['accuracy'].apply(lambda x: x.strip('%')) 

#cast column as float 
df['accuracy'] = df['accuracy'].astype(float)

*Percentages show that some values over 100 and some in negative these are going to be interesting to explore later for now I am just casting as straight float above 100 would be greater than 100%.*

In [63]:
#sanity check 
df.dtypes

meterID              object
testYear              int64
installationYear      int64
meterSize           float64
meterMake            object
accuracy            float64
dtype: object

*Next check any NAN values*

In [64]:
#null check
df.isna().sum()

meterID             2
testYear            0
installationYear    0
meterSize           0
meterMake           0
accuracy            0
dtype: int64

In [67]:
#view the nans
df[df.isnull().any(axis=1)]

,meterID,testYear,installationYear,meterSize,meterMake,accuracy
160,NaN,2018,2002,0.625,BADGER,98.64
221,NaN,2018,1990,0.625,BADGER,98.58


*Without a meterID (unique identifier there is no way to note which meter's these are. Since the NaNs in the dataset are only 2 of 1,962 total rows it represents only a very small percent of the dataset. With greater familiarity with the number convention you could perhaps assign these two new (stand in) numbers but for this exercise I feel time will be better spent exploring - so will drop and move on*

In [72]:
#drop nans 
df.dropna(inplace=True)

In [73]:
#sanity check
df.isna().sum()

meterID             0
testYear            0
installationYear    0
meterSize           0
meterMake           0
accuracy            0
dtype: int64

In [74]:
#descriptive statistics
df.describe()

,testYear,installationYear,meterSize,accuracy
count,1960.000000,1960.000000,1960.000000,1960.000000
mean,2018.537755,1997.148469,0.632270,95.716459
std,0.522689,14.311595,0.051255,13.841620
min,2018.000000,1941.000000,0.625000,-3.900000
25%,2018.000000,1994.000000,0.625000,97.910000
50%,2019.000000,2000.000000,0.625000,99.030000
75%,2019.000000,2006.000000,0.625000,99.780000
max,2020.000000,2019.000000,1.000000,102.350000


Overall working data comprises of:
* 1,960 rows
* 6 columns (5 + 1 identifier)  
* 4 numeric columns - meterSize & testYear & InstallationYear categorical / accuracy continuous  
* 1 string column meterMake categorical

Numeric Column Insights
* testYear - ranges from 2018 - 2020 mean
* installationYear - ranges from 1941 - 2019 with the mean meaning 1997 (long tail data)
* meterSize - ranges from 0.625 (5/8th of an inch) to 1 inch average is slightly bigger than 5/8th of an inch so they trend small  
* accuracy - ranges from -3.9 to 102.35. Mean is 95.72 so a large portion of the meter's are measuring at above 95% accurate. 

*Walking through the descriptive statistics helps me get a big picture of the the dataset and begin brainstorming where I want to go next with the data. Current thoughts:* 
* *accuracy - this is the number of gallons out of 100 gallon sample that would be read correctly. Above 100 should be flagged as inaccurate*
* *what is the cost per gallon? bring in outside data to understand revenue loss here* 
* *installationYear being low correlated to lower accuracy?* 
* *meterSize have any affect on accuracy?*
* *do the meter ids have any geographical correlation?* 
* *what is the total spent replacing per year per brand?* 
* *what is the correlation between accuracy and brand?* 
* *what is the correlation between installationYear and brand?*  
* *within a brand what sizes 'work' best?* 

# Auto Screen for Potentially Erroneous Reads 
The accuracy of the meter based on bench test results collected in the test year. The test results indicate the proportion of volume captured by the meter. For example, if the accuracy is 96% and 100 gallons of water were delivered to the customer, only 96 gallons would have been recorded by the meter and 4 gallons would be considered a revenue loss. Given this definition accuracy rates above 100% or below 0% may be miss functioning. Auto screen would be delivered to client to flag that meter reader (person or machine) may be delivering inaccurate results.

In [159]:
#auto flag accuracy above 100/ below 0 to  
accuracy_reader = (df['accuracy']<0) | (df['accuracy']>100)
df.loc[accuracy_reader]

,meterID,testYear,installationYear,meterSize,meterMake,accuracy
6,87593841,2018,2009,0.625,NEPTUNE,100.58
8,94206286,2018,1994,0.625,BADGER,100.13
18,42949864,2019,1997,0.625,NEPTUNE,100.65
28,90423750,2019,1990,0.625,BADGER,100.04
38,84065879,2019,2007,0.625,NEPTUNE,100.10
...,...,...,...,...,...,...
1946,74230546,2019,2000,0.625,NEPTUNE,100.14
1949,80862866,2019,2005,0.625,NEPTUNE,100.12
1950,17729077,2019,2017,0.625,BADGER,100.98
1957,8781150,2019,1958,0.625,TRIDENT,100.39


*This flags 343 of the rows as potentially miss read data. Having this insight would allow them to catch inaccurate measurement with little effort. - Value add*

# Brand Based Patterns in Age, Size & Accuracy 
Exploration of the distribution of brand overall in data set. Exploration into the correlation between brand age/size/accuracy and the intra brand trends.

*Brands seemed like the most straight forward of the areas to start diving into the dataset. It has only 3 categories and will potentially have high impact insights (i.e. this brand has the highest average accuracy scores)*

In [81]:
#count of each model
df['meterMake'].value_counts()

NEPTUNE    1373
BADGER      449
TRIDENT     138
Name: meterMake, dtype: int64

In [119]:
#display pie chart on type count
fig = go.Figure(data=[go.Pie(labels=['Neptune', 'Badger', 'Trident'],
                values=[1373, 449, 138])])   
fig.update_layout(font_family=font_family)
fig.update_traces(hoverinfo='label+percent', textinfo='label+ value+percent',
                  textfont_size=15, marker=dict(colors=colors, line=dict(color='#000000', width=1)))
fig.show()

*The vast majority of the pumps in this data set are 'Neptune' model, for any generalized models later may need to re sample to deal with data distribution imbalance.* 

In [142]:
#segment the df based on the maker type
badger = df.loc[df['meterMake'] == 'BADGER'] 
trident = df.loc[df['meterMake'] == 'TRIDENT']
neptune = df.loc[df['meterMake'] == 'NEPTUNE']

## Neptune 

In [143]:
#sanity check 
neptune.head()

,meterID,testYear,installationYear,meterSize,meterMake,accuracy
0,35826455,2019,1991,0.625,NEPTUNE,98.72
1,85381440,2018,2007,0.625,NEPTUNE,0.00
2,43165289,2019,1997,0.625,NEPTUNE,98.95
3,42947481,2019,1997,0.625,NEPTUNE,99.86
5,42721343,2019,1997,0.625,NEPTUNE,98.66


In [149]:
#descriptive statistics
neptune.describe()

,testYear,installationYear,meterSize,accuracy
count,1373.000000,1373.000000,1373.000000,1373.000000
mean,2018.518572,1999.946103,0.627003,95.976409
std,0.508511,8.407879,0.026923,11.969708
min,2018.000000,1963.000000,0.625000,-3.900000
25%,2018.000000,1995.000000,0.625000,97.800000
50%,2019.000000,2000.000000,0.625000,98.870000
75%,2019.000000,2007.000000,0.625000,99.520000
max,2020.000000,2013.000000,1.000000,102.350000


# Meter Size Patterns

In [128]:
def scatter_plot(df, x, y):
    fig = px.scatter(df, x=x, y=y)
    fig.show()

In [130]:
scatter_plot(df, x="accuracy", y="meterSize")

In [131]:
scatter_plot(df, x='installationYear', y="meterSize")

In [132]:
scatter_plot(df, x='testYear', y='meterSize')

In [133]:
scatter_plot(df, x='meterMake', y='meterSize')

# Data To Dos
* *accuracy - this is the number of gallons out of 100 gallon sample that would be read correctly. Above 100 should be flagged as inaccurate*
* *what is the cost per gallon? bring in outside data to understand revenue loss here* 
* *installationYear being low correlated to lower accuracy?* 
* *meterSize have any affect on accuracy?*
* *do the meter ids have any geographical correlation?* 
* *what is the total spent replacing per year per brand?* 
* *what is the correlation between accuracy and brand?* 
* *what is the correlation between installationYear and brand?*  
* *within a brand what sizes 'work' best?*